## GroupBy技术
Hadley Wickham‘s **spilt-apply-combine**

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
df = DataFrame({'key1':['a','a','b','b','a'],
               'key2':['one','two','one','two','one'],
               'data1':np.random.randn(5),
               'data2':np.random.randn(5)})
df

,data1,data2,key1,key2
0,0.187262,-0.691001,a,one
1,-0.933296,-0.746358,a,two
2,-0.388630,0.349054,b,one
3,0.328303,-0.256219,b,two
4,-0.611299,-0.538350,a,one


In [4]:
grouped = df['data1'].groupby(df['key1'])  # -split
grouped

In [11]:
grouped.mean()  # -apply-combine

key1
a    0.743114
b   -1.250000
Name: data1, dtype: float64

In [12]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.824102
      two     0.581137
b     one    -0.737151
      two    -1.762849
Name: data1, dtype: float64

In [13]:
# 多层次索引Series可以转成DataFrame
means.unstack()

key2,one,two
key1,,
a,0.824102,0.581137
b,-0.737151,-1.762849


In [3]:
states = np.array(['Ohio','California','California','Ohio','Ohio'])
years = np.array([2005,2005,2006,2005,2006])
df.data1.groupby([states,years]).mean()

California  2005   -0.675715
            2006   -0.183245
Ohio        2005   -0.594602
            2006    0.888925
Name: data1, dtype: float64

In [6]:
df.groupby('key1').mean()  # 用内部的列名作为分组键, 非数值列的麻烦列会被过滤

,data1,data2
key1,,
a,-0.245438,-0.466417
b,-0.211463,-0.993751


In [7]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one  -0.030300  0.089931
     two  -0.675715 -1.579114
b    one  -0.183245 -2.099175
     two  -0.239681  0.111674

In [8]:
df.groupby(['key1','key2']).size()  # 用size计数

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

groupby将缺失值NA自动剔除

---------------------------------------------
## 对分组迭代

In [9]:
# GROUPBY对象支持迭代，可以产生一组二元院组(由分组名和数据块组成)
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
      data1     data2 key1 key2
0 -0.949524 -1.704077    a  one
1 -0.675715 -1.579114    a  two
4  0.888925  1.883940    a  one
b
      data1     data2 key1 key2
2 -0.183245 -2.099175    b  one
3 -0.239681  0.111674    b  two


In [11]:
# 多重键，元组的第一个元素将会由键值组成的元组
for (k1, k2), group in df.groupby(['key1','key2']):
    print(k1, k2)
    print(group)

a one
      data1     data2 key1 key2
0 -0.949524 -1.704077    a  one
4  0.888925  1.883940    a  one
a two
      data1     data2 key1 key2
1 -0.675715 -1.579114    a  two
b one
      data1     data2 key1 key2
2 -0.183245 -2.099175    b  one
b two
      data1     data2 key1 key2
3 -0.239681  0.111674    b  two


In [3]:
# 对部分数据做成字典
pieces = dict(list(df.groupby('key1')))
pieces['b']

,data1,data2,key1,key2
2,-0.388630,0.349054,b,one
3,0.328303,-0.256219,b,two


In [5]:
# 用get_group命令效果是一样的，且更好
df.groupby('key1').get_group('b')  

,data1,data2,key1,key2
2,-0.388630,0.349054,b,one
3,0.328303,-0.256219,b,two


In [16]:
# 也可以换坐标轴
df.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [17]:
grouped = df.groupby(df.dtypes, axis = 1) # 默认按行分组
dict(list(grouped))

{dtype('float64'):       data1     data2
 0 -0.949524 -1.704077
 1 -0.675715 -1.579114
 2 -0.183245 -2.099175
 3 -0.239681  0.111674
 4  0.888925  1.883940, dtype('O'):   key1 key2
 0    a  one
 1    a  two
 2    b  one
 3    b  two
 4    a  one}

In [22]:
type(df['data1'])  # 降级成为Series

pandas.core.series.Series

In [21]:
type(df[['data1']])  # 保住DataFrame

pandas.core.frame.DataFrame

In [24]:
# 语法糖：
df.groupby('key1')['data1']  # df['data1'].groubpy(df['key1])

In [27]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.089931
     two  -1.579114
b    one  -2.099175
     two   0.111674

In [29]:
s_grouped = df.groupby(['key1','key2'])[['data2']]
s_grouped
s_grouped.mean()

data2
key1 key2          
a    one   0.089931
     two  -1.579114
b    one  -2.099175
     two   0.111674

In [38]:
# 通过字典或Series进行分组 -- 传入字典或者Series
people = DataFrame(np.random.randn(5,5),
                  columns = ['a', 'b', 'c', 'd', 'e'],
                   index = ['Joe','Steve','Wes','Jim','Travis'])
people.ix[2:3, ['b','c']] = np.nan  # ix是同时选取行和列
people

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """


,a,b,c,d,e
Joe,-0.529824,-0.984022,1.390756,0.230440,0.317197
Steve,1.136987,-0.354347,0.420992,1.115711,0.769670
Wes,-1.235050,NaN,NaN,-0.350951,-0.641152
Jim,-0.029870,-0.099889,0.332565,-0.217687,0.870894
Travis,-1.981474,0.206606,-0.061689,0.998659,-1.115891


In [39]:
mapping = {'a':'red','b':'red','c':'blue',
          'd':'blue','e':'red','f':'orange'}
by_column = people.groupby(mapping, axis = 1)
by_column.sum()

,blue,red
Joe,1.621195,-1.196649
Steve,1.536702,1.552310
Wes,-0.350951,-1.876201
Jim,0.114878,0.741136
Travis,0.936970,-2.890759


In [40]:
map_series = Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [41]:
people.groupby(map_series, axis = 1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [45]:
# 通过函数进行分组
people.groupby(len).sum()

,a,b,c,d,e
3,-1.794743,-1.083911,1.723321,-0.338198,0.546939
5,1.136987,-0.354347,0.420992,1.115711,0.769670
6,-1.981474,0.206606,-0.061689,0.998659,-1.115891


In [47]:
# 根据索引级别分组
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],
                                    [1,3,5,1,3,]], names = ['city','tenor'])
hier_df = DataFrame(np.random.randn(4,5),columns = columns)
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0     -0.602000  1.154678  0.390004  0.585525  0.127207
1      1.382817 -1.417822 -1.208256 -1.067149  0.126367
2      1.513536 -1.393984  0.311135 -0.709562  2.112948
3     -0.310221  0.034470  0.045998 -0.737540 -0.764148

In [49]:
hier_df.groupby(level = 'city', axis = 1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [50]:
hier_df.groupby(level = 'tenor', axis = 1).count()

tenor,1,3,5
0,2,2,1
1,2,2,1
2,2,2,1
3,2,2,1


## 数据聚合
* 内置的Series方法，如mean,count,min,sum,quantile,median,std,var,prod,fisrt,last
* agg/aggregate(自定义函数）

In [51]:
def  peak_to_peak(arr):
    return arr.max()-arr.min()
grouped = df.groupby('key1')
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.838449,3.588017
b,0.056436,2.210849


In [55]:
# 有些方法也可以用， 如describe
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.245438  0.991881 -0.949524 -0.812620 -0.675715  0.106605   
b      2.0 -0.211463  0.039906 -0.239681 -0.225572 -0.211463 -0.197354   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.888925   3.0 -0.466417  2.036428 -1.704077 -1.641596 -1.579114   
b    -0.183245   2.0 -0.993751  1.563306 -2.099175 -1.546463 -0.993751   

                          
           75%       max  
key1                      
a     0.152413  1.883940  
b    -0.441038  0.111674

In [56]:
tips = pd.read_csv('tips.csv')
tips['tip_pct'] = tips.tip/tips.total_bill
tips[:6]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240


In [57]:
# 对不同列用不同方法
grouped = tips.groupby(['sex','smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

sex     smoker
Female  No        0.156921
        Yes       0.182150
Male    No        0.160669
        Yes       0.152771
Name: tip_pct, dtype: float64

In [58]:
# 一次传入一组函数
grouped_pct.agg(['mean','std',peak_to_peak])  # 默认列名是函数名

mean       std  peak_to_peak
sex    smoker                                  
Female No      0.156921  0.036421      0.195876
       Yes     0.182150  0.071595      0.360233
Male   No      0.160669  0.041849      0.220186
       Yes     0.152771  0.090588      0.674707

In [59]:
# 可以传入自定义列名
grouped_pct.agg([('foo','mean'),('bar',np.std)])

foo       bar
sex    smoker                    
Female No      0.156921  0.036421
       Yes     0.182150  0.071595
Male   No      0.160669  0.041849
       Yes     0.152771  0.090588

In [61]:
functions = ['count','mean','max']
result = grouped['tip_pct','total_bill'].agg(functions)
result

tip_pct                     total_bill                  
                count      mean       max      count       mean    max
sex    smoker                                                         
Female No          54  0.156921  0.252672         54  18.105185  35.83
       Yes         33  0.182150  0.416667         33  17.977879  44.30
Male   No          97  0.160669  0.291990         97  19.791237  48.33
       Yes         60  0.152771  0.710345         60  22.284500  50.81

In [62]:
result.tip_pct

count      mean       max
sex    smoker                           
Female No         54  0.156921  0.252672
       Yes        33  0.182150  0.416667
Male   No         97  0.160669  0.291990
       Yes        60  0.152771  0.710345

In [67]:
result.tip_pct[['mean']]  # 还可以单

mean
sex    smoker          
Female No      0.156921
       Yes     0.182150
Male   No      0.160669
       Yes     0.152771

In [68]:
# 定向精确 -- 还是用字典
grouped.agg({'tip':np.max, 'size':'sum'})

tip  size
sex    smoker            
Female No       5.2   140
       Yes      6.5    74
Male   No       9.0   263
       Yes     10.0   150

In [69]:
grouped.agg({'tip_pct':['min','max','mean','std'],
            'size':'sum'})

size   tip_pct                              
               sum       min       max      mean       std
sex    smoker                                             
Female No      140  0.056797  0.252672  0.156921  0.036421
       Yes      74  0.056433  0.416667  0.182150  0.071595
Male   No      263  0.071804  0.291990  0.160669  0.041849
       Yes     150  0.035638  0.710345  0.152771  0.090588

In [70]:
# 把没必要的轴信息去掉，无索引形式返回
tips.groupby(['sex','smoker'], as_index = False).mean()

,sex,smoker,total_bill,tip,size,tip_pct
0,Female,No,18.105185,2.773519,2.592593,0.156921
1,Female,Yes,17.977879,2.931515,2.242424,0.182150
2,Male,No,19.791237,3.113402,2.711340,0.160669
3,Male,Yes,22.284500,3.051167,2.500000,0.152771


In [72]:
people

,a,b,c,d,e
Joe,-0.529824,-0.984022,1.390756,0.230440,0.317197
Steve,1.136987,-0.354347,0.420992,1.115711,0.769670
Wes,-1.235050,NaN,NaN,-0.350951,-0.641152
Jim,-0.029870,-0.099889,0.332565,-0.217687,0.870894
Travis,-1.981474,0.206606,-0.061689,0.998659,-1.115891


In [77]:
key = ['1','2','1','2','1']
people.groupby(key).mean()

,a,b,c,d,e
1,-1.248782,-0.388708,0.664533,0.292716,-0.479949
2,0.553559,-0.227118,0.376778,0.449012,0.820282


In [78]:
people.groupby(key).transform(np.mean)  # 卧槽！

,a,b,c,d,e
Joe,-1.248782,-0.388708,0.664533,0.292716,-0.479949
Steve,0.553559,-0.227118,0.376778,0.449012,0.820282
Wes,-1.248782,-0.388708,0.664533,0.292716,-0.479949
Jim,0.553559,-0.227118,0.376778,0.449012,0.820282
Travis,-1.248782,-0.388708,0.664533,0.292716,-0.479949


In [79]:
# 距平化函数-利用transform的广播
def demean(arr):
    return arr - arr.mean()
demeaned = people.groupby(key).transform(demean)
demeaned

,a,b,c,d,e
Joe,0.718959,-0.595314,0.726222,-0.062276,0.797145
Steve,0.583429,-0.127229,0.044213,0.666699,-0.050612
Wes,0.013733,NaN,NaN,-0.643667,-0.161203
Jim,-0.583429,0.127229,-0.044213,-0.666699,0.050612
Travis,-0.732691,0.595314,-0.726222,0.705943,-0.635942


In [80]:
demeaned.groupby(key).mean()

,a,b,c,d,e
1,0.0,0.000000e+00,-5.551115e-17,0.0,-3.700743e-17
2,0.0,1.387779e-17,0.000000e+00,0.0,0.000000e+00


## apply: 一般性的 “ 拆分-应用-合并 ”
* transform:传入的函数只能生产两种结果，1.产生一个可以广播的标量值（np.mean), 2.产生一个相同大小的结果数组
* 最一般化的GroupBy的方法是apply
> apply会待处理的对象拆成多个片段，然后各片段调用传入的函数。再合并

In [82]:
def top(df, n = 5, column = 'tip_pct'):
    return df.sort_values(by = column)[-n:]
top(tips, n = 6)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [83]:
# 使用apply -- 可以传入任何函数，只需要返回一个标量或者pandas对象
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

In [85]:
tips.groupby(['smoker','day']).apply(top, n = 1, column = 'total_bill')  # 传入apply的函数额外参数放在后面

total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
No     Fri  94        22.75   3.25  Female     No   Fri  Dinner     2   
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4   
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6   
       Thur 142       41.19   5.00    Male     No  Thur   Lunch     5   
Yes    Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4   
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3   
       Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4   

                  tip_pct  
smoker day                 
No     Fri  94   0.142857  
       Sat  212  0.186220  
       Sun  156  0.103799  
       Thur 142  0.121389  
Yes    Fri  95   0.117750  
       Sat  170  0.196812  
       Sun  182  0.077178  
       Thur 197  0.115982

In [86]:
# 禁止分组键 -- 还是那么分组，但是不产生层次化索引
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

In [87]:
tips.groupby('smoker', group_keys = False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [88]:
# 分位数和桶分析
frame = DataFrame({'data1':np.random.randn(1000),
                  'data2':np.random.randn(1000)})
factor = pd.cut(frame.data1, 4)
factor[:10] 

0    (0.0994, 1.548]
1    (0.0994, 1.548]
2    (-1.35, 0.0994]
3    (0.0994, 1.548]
4    (-1.35, 0.0994]
5    (0.0994, 1.548]
6    (-2.804, -1.35]
7    (0.0994, 1.548]
8    (0.0994, 1.548]
9    (0.0994, 1.548]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.804, -1.35] < (-1.35, 0.0994] < (0.0994, 1.548] < (1.548, 2.997]]

In [89]:
def get_stats(group):
    return {'min':group.min(),'max':group.max(),
           'count':group.count(), 'mean':group.mean()}
grouped = frame.data2.groupby(factor)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-2.804, -1.35]",90.0,2.220941,-0.130118,-2.738312
"(-1.35, 0.0994]",445.0,3.456392,0.009237,-2.790257
"(0.0994, 1.548]",396.0,2.821088,0.016371,-3.224087
"(1.548, 2.997]",69.0,1.701761,-0.184675,-2.640746


In [90]:
grouping = pd.qcut(frame.data1, 10, labels = False)
grouped

data1                 
(-2.804, -1.35]  count     90.000000
                 max        2.220941
                 mean      -0.130118
                 min       -2.738312
(-1.35, 0.0994]  count    445.000000
                 max        3.456392
                 mean       0.009237
                 min       -2.790257
(0.0994, 1.548]  count    396.000000
                 max        2.821088
                 mean       0.016371
                 min       -3.224087
(1.548, 2.997]   count     69.000000
                 max        1.701761
                 mean      -0.184675
                 min       -2.640746
Name: data2, dtype: float64

## pivot table & cross table

In [92]:
tips.pivot_table(index = ['sex', 'smoker'])

size       tip   tip_pct  total_bill
sex    smoker                                          
Female No      2.592593  2.773519  0.156921   18.105185
       Yes     2.242424  2.931515  0.182150   17.977879
Male   No      2.711340  3.113402  0.160669   19.791237
       Yes     2.500000  3.051167  0.152771   22.284500

In [94]:
tips.pivot_table(['tip_pct','size'], index = ['sex', 'day'], columns = 'smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
sex    day                                         
Female Fri   2.500000  2.000000  0.165296  0.209129
       Sat   2.307692  2.200000  0.147993  0.163817
       Sun   3.071429  2.500000  0.165710  0.237075
       Thur  2.480000  2.428571  0.155971  0.163073
Male   Fri   2.000000  2.125000  0.138005  0.144730
       Sat   2.656250  2.629630  0.162132  0.139067
       Sun   2.883721  2.600000  0.158291  0.173964
       Thur  2.500000  2.300000  0.165706  0.164417

In [96]:
tips.pivot_table(['tip_pct','size'], index = ['sex', 'day'],
                columns = 'smoker', margins = True)  # 加了个all

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
sex    day                                                             
Female Fri   2.500000  2.000000  2.111111  0.165296  0.209129  0.199388
       Sat   2.307692  2.200000  2.250000  0.147993  0.163817  0.156470
       Sun   3.071429  2.500000  2.944444  0.165710  0.237075  0.181569
       Thur  2.480000  2.428571  2.468750  0.155971  0.163073  0.157525
Male   Fri   2.000000  2.125000  2.100000  0.138005  0.144730  0.143385
       Sat   2.656250  2.629630  2.644068  0.162132  0.139067  0.151577
       Sun   2.883721  2.600000  2.810345  0.158291  0.173964  0.162344
       Thur  2.500000  2.300000  2.433333  0.165706  0.164417  0.165276
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [97]:
tips.pivot_table('tip_pct', index = ['sex','smoker'], columns = 'day',
                aggfunc = len, margins = True)  # 还可以聚合函数

day             Fri   Sat   Sun  Thur    All
sex    smoker                               
Female No       2.0  13.0  14.0  25.0   54.0
       Yes      7.0  15.0   4.0   7.0   33.0
Male   No       2.0  32.0  43.0  20.0   97.0
       Yes      8.0  27.0  15.0  10.0   60.0
All            19.0  87.0  76.0  62.0  244.0

In [98]:
tips.pivot_table('size', index = ['time','sex','smoker'],
                columns = 'day', aggfunc = 'sum', fill_value = 0)

day                   Fri  Sat  Sun  Thur
time   sex    smoker                     
Dinner Female No        2   30   43     2
              Yes       8   33   10     0
       Male   No        4   85  124     0
              Yes      12   71   39     0
Lunch  Female No        3    0    0    60
              Yes       6    0    0    17
       Male   No        0    0    0    50
              Yes       5    0    0    23

In [99]:
# 交叉表：一种用于计算分组频率的特殊透视表
pd.crosstab([tips.time, tips.day], tips.smoker, margins = True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244